# Print Metrics

## Functions

In [34]:
from pathlib import Path
import os
import pandas as pd

In [35]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


In [36]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    # Print columns in the CSV
    # print("\n========== Columns in CSV ==========")
    # print(df.columns)

    # Print final metrics
    print_final_metrics(df)


In [37]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    # Load both result CSVs
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)
    # print(df1.head())
    # Use the final row (last epoch)
    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("🔍 Comparison of Final Epoch Metrics:\n")
    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        trend = "✅ Good Increase" if val2 > val1 else "❌ No Increase"
        print(f"{label:20s}: {val1:.5f} → {val2:.5f} | {trend}")

In [38]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\n📊 Comparison of mAP@0.5:0.95 per class:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Change'}")
    print("-" * 50)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        if map2 > map1:
            status = "✅ Good increase"
        else:
            status = "❌ No increase"

        print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {status}")


In [39]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model):
    return model.val()

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("⚠️ No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"✅ Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)



In [40]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("📊 Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


## Calling Functions

### Before Retraning

In [41]:
yolov8 = './runs/train/yolov8'
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8\results.csv
Total number of epochs: 78

========== Final Training Metrics ==========
Training Loss: 1.752180
Precision: 0.592850
Recall: 0.331840
mAP@0.5: 0.401860
mAP@0.5:0.95: 0.280340

========== Final Validation Metrics ==========
Validation Loss: 5.269870


In [42]:
best_pt_path = find_best_model(yolov8)
# evaluate_and_save_metrics(best_pt_path)

✅ Found best.pt at: runs\train\yolov8\weights\best.pt


In [43]:
print_per_class_metrics("per_class_metrics.json")

📊 Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.2943         
pool            1          0.8215         
vegetation      2          0.1216         
roof            3          0.5337         
wall            4          0.1327         
window          5          0.1891         
person          6          0.2558         
dog             7          0.4714         
car             8          0.3195         
bicycle         9          0.2275         
tree            10         0.4541         
truck           11         0.0            
bus             12         0.0            
vehicle         13         0.2993         


### After Retraning

In [44]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8\results.csv
Total number of epochs: 30

========== Final Training Metrics ==========
Training Loss: 1.764910
Precision: 0.817130
Recall: 0.755970
mAP@0.5: 0.851300
mAP@0.5:0.95: 0.744380

========== Final Validation Metrics ==========
Validation Loss: 2.098990


In [45]:
best_pt_path = find_best_model(new_path)
# evaluate_and_save_metrics(best_pt_path, output_json_path="per_class_metrics_retrain.json")

✅ Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt


In [46]:
print_per_class_metrics("per_class_metrics_retrain.json")

📊 Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
road            0          0.7454         
pool            1          0.7454         
vegetation      2          0.7454         
roof            3          0.7915         
wall            4          0.8921         
window          5          0.7584         
person          6          0.6346         
dog             7          0.7454         
car             8          0.7454         
bicycle         9          0.5281         
tree            10         0.8665         
truck           11         0.7454         
bus             12         0.7454         
vehicle         13         0.7467         


### Compare both metrics

In [47]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)

compare_final_metrics(results_csv_path_1, results_csv_path)

🔍 Comparison of Final Epoch Metrics:

Box Loss (Train)    : 0.58367 → 0.54170 | ❌ No Increase
Cls Loss (Train)    : 0.32681 → 0.38641 | ✅ Good Increase
DFL Loss (Train)    : 0.84170 → 0.83680 | ❌ No Increase
Precision           : 0.59285 → 0.81713 | ✅ Good Increase
Recall              : 0.33184 → 0.75597 | ✅ Good Increase
mAP@0.5             : 0.40186 → 0.85130 | ✅ Good Increase
mAP@0.5:0.95        : 0.28034 → 0.74438 | ✅ Good Increase
Box Loss (Val)      : 1.78693 → 0.54063 | ❌ No Increase
Cls Loss (Val)      : 2.43818 → 0.70702 | ❌ No Increase
DFL Loss (Val)      : 1.04476 → 0.85134 | ❌ No Increase


In [48]:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")


📊 Comparison of mAP@0.5:0.95 per class:

Class           Before     After      Change
--------------------------------------------------
unlabeled       0.2943     0.0000     ❌ No increase
pool            0.8215     0.7454     ❌ No increase
vegetation      0.1216     0.7454     ✅ Good increase
roof            0.5337     0.7915     ✅ Good increase
wall            0.1327     0.8921     ✅ Good increase
window          0.1891     0.7584     ✅ Good increase
person          0.2558     0.6346     ✅ Good increase
dog             0.4714     0.7454     ✅ Good increase
car             0.3195     0.7454     ✅ Good increase
bicycle         0.2275     0.5281     ✅ Good increase
tree            0.4541     0.8665     ✅ Good increase
truck           0.0000     0.7454     ✅ Good increase
bus             0.0000     0.7454     ✅ Good increase
vehicle         0.2993     0.7467     ✅ Good increase
